<a href="https://colab.research.google.com/github/noa-blenkitny/IR_Project/blob/main/search_backend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Authenticate your user
# The authentication should be done with the email connected to your GCP account
from google.colab import auth
auth.authenticate_user()

In [2]:
# Copy one wikidumps files 
import os
from pathlib import Path
from google.colab import auth

project_id = 'core-period-321814'
!gcloud config set project {project_id}

data_bucket_name = 'wikidata_preprocessed'
try:
    if os.environ["wikidata_preprocessed"] is not None:
        pass  
except:
      !mkdir wikidumps
      !gsutil cp gs://{data_bucket_name}/multistream1_preprocessed.parquet "wikidumps/" 

Updated property [core/project].
mkdir: cannot create directory ‘wikidumps’: File exists
Copying gs://wikidata_preprocessed/multistream1_preprocessed.parquet...
| [1 files][316.7 MiB/316.7 MiB]                                                
Operation completed over 1 objects/316.7 MiB.                                    


In [3]:
!pip install -q pyspark
!pip install -U -q PyDrive
!apt-get update -qq
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *
# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
#sc.stop()
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

openjdk-8-jdk-headless is already the newest version (8u312-b07-0ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 100 not upgraded.
--2022-01-02 10:27:02--  https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar
Resolving repos.spark-packages.org (repos.spark-packages.org)... 13.32.87.68, 13.32.87.63, 13.32.87.119, ...
Connecting to repos.spark-packages.org (repos.spark-packages.org)|13.32.87.68|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘/usr/local/lib/python3.7/dist-packages/pyspark/jars/graphframes-0.8.2-spark3.2-s_2.12.jar’ not modified on server. Omitting download.



In [4]:
#when we move to gcp- nedd to change it to the entire wikipdia corpus
from pathlib import Path 
import os

try:
    if os.environ["wikidata_preprocessed"] is not None:
      path = os.environ["wikidata_preprocessed"]+"/wikidumps/*"
except:
      path = "wikidumps/*"

parquetFile = spark.read.parquet(path)
parquetFile.show()

+---+--------------------+--------------------+--------------------+
| id|               title|                text|         anchor_text|
+---+--------------------+--------------------+--------------------+
| 12|           Anarchism|'''Anarchism''' i...|[{23040, politica...|
| 25|              Autism|'''Autism''' is a...|[{492271, Clinica...|
| 39|              Albedo|thumb|upright=1.3...|[{679294, diffuse...|
|290|                   A|'''A''', or '''a'...|[{290, See below}...|
|303|             Alabama|'''Alabama''' () ...|[{351590, Yellowh...|
|305|            Achilles|thumb|260px|Ancie...|[{1076007, potter...|
|307|     Abraham Lincoln|'''Abraham Lincol...|[{1827174, Alexan...|
|308|           Aristotle|'''Aristotle''' (...|[{1389981, bust},...|
|309|An American in Paris|'''''An American ...|[{13066, George G...|
|316|Academy Award for...|The '''Academy Aw...|[{39842, Academy ...|
|324|      Academy Awards|The '''Academy Aw...|[{649481, film in...|
|330|             Actrius|'''''Act

In [5]:
docs_list = [23862,7089,21035]
body_data = parquetFile.select("id","text").rdd.filter(lambda x: x[0] in docs_list)
title_data = parquetFile.select("id","title").rdd.filter(lambda x: x[0] in docs_list)
raw_anchor_data = parquetFile.select("id","anchor_text").rdd.filter(lambda x: x[0] in docs_list)
#using set because eventually the anchor index will use binary ranking and we want unique tokens.
#filtering only the text of anchor because we think that if a doc points to another in a specific topic the
#doc is more imprtant. we will use it in the binary calculations of search score
anchor_data = raw_anchor_data.mapValues(lambda x: ','.join(set(item[1] for item in x)))
#print(body_data.take(3))
#print(title_data.take(3))
#print(anchor_data.take(1))

In [6]:
from IPython.core.display import Math
from inverted_index_colab import InvertedIndex
from inverted_index_colab import MultiFileReader
from inverted_index_colab import MultiFileWriter
import math

import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
import re
from collections import OrderedDict, Counter, defaultdict
import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')
#using code from assignment 3- gcp
#funcs to help build the inverted index
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def calc_doc_len(text, id):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [token for token in tokens if token not in all_stopwords]
  return len(tokens)

def word_count(text, id):
  
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [token for token in tokens if token not in all_stopwords ]
  word_count_list = []
  words_dict= OrderedDict()
  for word in tokens:
    words_dict[word] = words_dict.get(word,0) + 1
  for item in words_dict.items():
    word_count_list.append((item[0],(id,item[1])))
  return word_count_list

def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  return  sorted(unsorted_pl,key=lambda y: y[0])

def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''

  return postings.mapValues(lambda x : len(x))

def partition_postings_and_write(postings,new_index_name):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  #partitioning RDD to (bucket_id, [(w0, posting_list_0)) entries 
  postings = postings.map(lambda x: (new_index_name+str(token2bucket_id(x[0])),x))
  #reducing it to (bucket_id, [(w0, posting_list_0), (w1, posting_list_1), ...])
  postings = postings.groupByKey()
  #writing each bucket to disk
  #need to change it in gcp to:
  #postings = postings.map(lambda y: (InvertedIndex.write_a_posting_list(y,'3b1u8c3k6e4t122')))
  #in colab:
  postings = postings.map(lambda y: (InvertedIndex.write_a_posting_list(y)))
  return postings
  
def calc_tf_idf(tf, N , df, doc_len):
  tf_idf = (tf/doc_len)* math.log(N/df)
  return tf_idf



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
from time import time
def create_inverted_index_with_tf_idf_weights(data,new_index_name):

    # word counts map - posting lists
    word_counts = data.flatMap(lambda x: word_count(x[1], x[0]))
    #united posting lists
    postings = word_counts.groupByKey().mapValues(reduce_word_counts)
    #RDD to map doc_id to doc_len
    doc_lens = data.map(lambda x: (x[0], calc_doc_len(x[1], x[0])))
    # turn the doc_lens RDD into dict - TODO: check how its gonna be for the whole corpus
    doc_lens = doc_lens.collectAsMap()
    # print(doc_lens)
    # RDD to map {doc_id,[(token,tf),(token,tf)....]}
    tf = postings.flatMap(lambda x: [(tup[0], (x[0],tup[1])) for tup in x[1]] ) 
    # groupBy doc_id
    tf = tf.groupByKey().mapValues(list)
   
    # filtering postings and calculate df
    #Next, we will filter out rare words, words that appear in 10 or fewer documents (when working on the entire corpus, we will increase this threshold to a minimum of 50 documents).
    #postings_filtered = postings.filter(lambda x: len(x[1])>50) #why filter?
    #w2df = calculate_df(postings_filtered)
    
    w2df = calculate_df(postings)
    w2df_dict = w2df.collectAsMap()

    # partition posting lists and write out- in gcp
    #_ = partition_postings_and_write(postings_filtered).collect()
    #_ = partition_postings_and_write(postings_filtered)

    #in colab:
    #write posting lists to disk and returns posting locs dict for each bucket
    posting_locs_list = partition_postings_and_write(postings,new_index_name).collect()
    

    # collect all posting lists locations into one super-set- gcp code:
    # super_posting_locs = defaultdict(list)
    # for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
    #   if not blob.name.endswith("pickle"):
    #     continue
    #   with blob.open("rb") as f:
    #     posting_locs = pickle.load(f)
    #     for k, v in posting_locs.items():
    #       super_posting_locs[k].extend(v)

    # merge the posting locations into a single dict- good for local test in colab
    super_posting_locs = defaultdict(list)
    for posting_loc in posting_locs_list:
        for k, v in posting_loc.items():
          super_posting_locs[k].extend(v)

      #for k, v in posting_loc.items():
        #super_posting_locs[k].extend(v)

    new_index = InvertedIndex()
    # Adding the posting locations dictionary to the inverted index
    new_index.posting_locs = super_posting_locs
    # Add the token - df dictionary to the inverted index
    new_index.df = w2df_dict
    # the number of the docs in the corpus - using to calc TF-IDF
    new_index.N= data.count()
    # print(new_index.N)
    # tf = postings.flatMap(lambda x: [(tup[0], (x[0],tup[1])) for tup in x[1]] ) 
    tf_idf = tf.flatMap(lambda x: [(x[0],(tup[0], calc_tf_idf(tup[1], new_index.N , new_index.df[tup[0]], doc_lens[x[0]]))) for tup in x[1]] ) 
    tf_idf = tf_idf.groupByKey().mapValues(list)
    new_index.weights = tf_idf.collectAsMap()
    #print(new_index.tf_idf)
    # write the global stats out
    new_index.write_index('.', new_index_name)

    return new_index


In [8]:

def create_inverted_index_with_binary_weights(data,new_index_name):

    # word counts map - posting lists
    word_counts = data.flatMap(lambda x: word_count(x[1], x[0]))
    #united posting lists
    postings = word_counts.groupByKey().mapValues(reduce_word_counts)
    #RDD to map doc_id to doc_len
    #doc_lens = data.map(lambda x: (x[0], calc_doc_len(x[1], x[0])))
    # turn the doc_lens RDD into dict - TODO: check how its gonna be for the whole corpus
    #doc_lens = doc_lens.collectAsMap()
    # print(doc_lens)
    # RDD to map {doc_id,[(token,tf),(token,tf)....]}
    tf = postings.flatMap(lambda x: [(tup[0], (x[0],tup[1])) for tup in x[1]] ) 
    # groupBy doc_id
    tf = tf.groupByKey().mapValues(list)
   
    # filtering postings and calculate df
    #Next, we will filter out rare words, words that appear in 10 or fewer documents (when working on the entire corpus, we will increase this threshold to a minimum of 50 documents).
    #postings_filtered = postings.filter(lambda x: len(x[1])>50) #why filter?
    #w2df = calculate_df(postings_filtered)
    
    w2df = calculate_df(postings)
    w2df_dict = w2df.collectAsMap()

    # partition posting lists and write out- in gcp
    #_ = partition_postings_and_write(postings_filtered).collect()
    #_ = partition_postings_and_write(postings_filtered)

    #in colab:
    #write posting lists to disk and returns posting locs dict for each bucket
    posting_locs_list = partition_postings_and_write(postings,new_index_name).collect()
    

    # collect all posting lists locations into one super-set- gcp code:
    # super_posting_locs = defaultdict(list)
    # for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
    #   if not blob.name.endswith("pickle"):
    #     continue
    #   with blob.open("rb") as f:
    #     posting_locs = pickle.load(f)
    #     for k, v in posting_locs.items():
    #       super_posting_locs[k].extend(v)

    # merge the posting locations into a single dict- good for local test in colab
    super_posting_locs = defaultdict(list)
    for posting_loc in posting_locs_list:
        for k, v in posting_loc.items():
          super_posting_locs[k].extend(v)

      #for k, v in posting_loc.items():
        #super_posting_locs[k].extend(v)

    new_index = InvertedIndex()
    # Adding the posting locations dictionary to the inverted index
    new_index.posting_locs = super_posting_locs
    # Add the token - df dictionary to the inverted index
    new_index.df = w2df_dict
    # the number of the docs in the corpus - using to calc TF-IDF
    new_index.N= data.count()
    # print(new_index.N)
    # tf = postings.flatMap(lambda x: [(tup[0], (x[0],tup[1])) for tup in x[1]] ) 
    tf_idf = tf.flatMap(lambda x: [(x[0],(tup[0], 1)) for tup in x[1]] ) 
    tf_idf = tf_idf.groupByKey().mapValues(list)
    new_index.weights = tf_idf.collectAsMap()
    # write the global stats out
    new_index.write_index('.', new_index_name)

    return new_index

In [9]:

#creating body index:
# time the index creation time
#t_start = time()
#index_const_time = time() - t_start

#creating indexes- one for body, one for title and one for anchor text
body_index = create_inverted_index_with_tf_idf_weights(body_data,"body_index")
title_index = create_inverted_index_with_binary_weights(title_data,"title_index")
anchor_index =  create_inverted_index_with_binary_weights(anchor_data,"anchor_index")
    #docid len map
    #doc_len_map = data.Map(lambda x: calc_doc_len(x[1],x[0]))
    #print(doc_len_map)



In [10]:
#tests for us- to see that read index works, should also work on full curpos?
test_body= InvertedIndex.read_index('.', "body_index")
test_title = InvertedIndex.read_index('.', "title_index")
test_anchor = InvertedIndex.read_index('.', "anchor_index")
print(test_body.weights)
print(test_title.weights)
print(test_anchor.weights)



{7089: [('1855', 0.0002563257789706276), ('made', 0.0), ('cacao', 0.004870189800441923), ('available', 0.0), ('paste', 0.0005126515579412552), ('flavoring', 0.0002563257789706276), ('foods', 0.0007568177472854212), ('form', 0.0), ('majority', 0.0001892044368213553), ('mesoamerican', 0.0007689773369118827), ('beverages', 0.0006622155288747434), ('intense', 9.460221841067765e-05), ('must', 0.0003784088736427106), ('develop', 0.0), ('flavor', 0.0023069320107356483), ('dried', 0.0004730110920533882), ('removed', 0.0004730110920533882), ('cocoa', 0.04203742775118292), ('nibs', 0.0007689773369118827), ('liquor', 0.0041012124635300415), ('solids', 0.004357538242500669), ('varying', 0.0015379546738237655), ('proportions', 0.0010253031158825104), ('dutch', 0.0005676133104640659), ('combination', 0.0), ('foodstuffs', 0.0002563257789706276), ('particularly', 0.0), ('desserts', 0.0007689773369118827), ('sweetened', 0.0002563257789706276), ('eaten', 0.0002563257789706276), ('hearts', 0.000256325778

In [11]:
#posting list by word- could be useful when doing calculation dynamically in relation to the words in the query
TUPLE_SIZE = 6       
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer
from contextlib import closing

def read_posting_list(inverted, w):
  with closing(MultiFileReader()) as reader:
    locs = inverted.posting_locs[w]
    b = reader.read(locs, inverted.df[w] * TUPLE_SIZE)
    posting_list = []
    for i in range(inverted.df[w]):
      doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
      tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
      posting_list.append((doc_id, tf))
    return posting_list